In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp gevent_client

<IPython.core.display.Javascript object>

In [ ]:
# export

import gevent

from gevent import monkey

monkey.patch_all()


import time
import requests

from datetime import datetime

from will_it_saturate.servers import BaseServer
from will_it_saturate.clients import BaseClient
from will_it_saturate.registry import register_model


def fetch(url):
    response = requests.get(url)
    response.raise_for_status()
    started = datetime.now()
    stopped = datetime.now()
    response.started = started
    response.stopped = stopped
    return response


class FakeResponse:
    def __init__(self, url, content, started, stopped):
        self.url = url
        self.content = content
        self.started = started
        self.stopped = stopped


def streaming_fetch(url):
    chunks = []
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        started = datetime.now()
        for chunk in r.iter_content(chunk_size=4096):
            chunks.append(chunk)
    stopped = datetime.now()
    response = FakeResponse(url, b"".join(chunks), started, stopped)
    return response


@register_model
class GeventClient(BaseClient):
    timestamps = []

    def set_timestamps(self, responses):
        for response in responses:
            self.timestamps.append((response.started, response.stopped))

    def measure_server(self, epoch):
        print("measure server")
        print(epoch.urls[0])
        start = time.perf_counter()
        # jobs = [gevent.spawn(fetch, _url) for _url in epoch.urls]
        jobs = [gevent.spawn(streaming_fetch, _url) for _url in epoch.urls]
        responses = gevent.wait(jobs)
        elapsed = time.perf_counter() - start
        responses = [r.value for r in responses]
        return elapsed, responses

    def measure(self, epoch):
        print("measure")
        # monkey.patch_all()
        elapsed, responses = self.measure_server(epoch)
        self.verify_checksums(epoch, responses)
        self.set_timestamps(responses)
        return elapsed

<IPython.core.display.Javascript object>

In [ ]:
# export

import typer


from will_it_saturate.hosts import Host
from will_it_saturate.epochs import Epoch
from will_it_saturate.servers import BaseServer
from will_it_saturate.control.client import ControlClient


def run_gevent_with_args(exponent: int, server_host_name: str):
    print("running requests with gevent")
    typer.echo(f"exponent {exponent}, server_host_name: {server_host_name}")
    control_server_port, server_port = 8100, 5100
    server = BaseServer(host=server_host_name, port=server_port)
    server_control_host = Host(name=server_host_name, port=control_server_port)
    server_control_client = ControlClient(host=server_control_host)
    epoch = Epoch(file_size=10 ** exponent, duration=10)
    epoch.files = server_control_client.get_or_create_files(epoch)
    epoch.create_urls_from_files(server)
    benchmark_client = GeventClient(
        name="gevent", host=server_host_name, port=server_port
    )
    elapsed = benchmark_client.measure(epoch)
    print(f"elapsed: {elapsed}")


def run_gevent():
    typer.run(run_gevent_with_args)

<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_fastapi_main.ipynb.
Converted 12_django_views.ipynb.
Converted 13_django_handlers.ipynb.
Converted 14_django_http.ipynb.
Converted 15_django_asgi.ipynb.
Converted 15_django_async_settings.ipynb.
Converted 15_django_settings.ipynb.
Converted 15_django_urls.ipynb.
Converted 15_django_wsgi.ipynb.
Converted 16_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 22_gevent_client.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_fr

<IPython.core.display.Javascript object>